In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import IterativeImputer
import miceforest as mf
from missforest import MissForest
import MIDASpy as md
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
df = pd.read_excel(r"F:\Work stuff\Opthalmology\berlin\raw.xlsx", 'raw')

def prep(df: pd.DataFrame):
    """
    Preprocess the DataFrame by:
      - Dropping rows with missing values and resetting the index.
      - Converting object columns to categorical via LabelEncoder.
      - Converting other columns to float (and then to int if >50% of values are integer-like).
      - If any numeric column (not already marked as categorical) has only 2 unique values,
        it is considered categorical and encoded.
    
    Returns:
      categorical_cols (list): List of columns encoded as categorical.
      discrete_cols (list): List of columns that are numeric and integer-like.
      cont_cols (list): List of remaining continuous numeric columns.
      df_clean (DataFrame): The preprocessed DataFrame.
      encoders (dict): Mapping from categorical column name to its LabelEncoder.
    """
    # Drop rows with missing values and reset the index.
    df_clean = df.dropna().reset_index(drop=True)

    categorical_cols = []
    discrete_cols = []
    encoders = {}

    # Process each column.
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            # Mark as categorical and encode using LabelEncoder.
            categorical_cols.append(col)
            le = LabelEncoder()
            df_clean[col] = le.fit_transform(df_clean[col])
            encoders[col] = le
        else:
            try:
                # Convert column to float.
                df_clean[col] = df_clean[col].astype(float)
                # If >50% of values are integer-like, cast column to int.
                if (np.isclose(df_clean[col] % 1, 0).mean() > 0.5):
                    df_clean[col] = df_clean[col].astype(int)
                    discrete_cols.append(col)
            except (ValueError, TypeError):
                # If conversion fails, treat the column as categorical.
                categorical_cols.append(col)
                le = LabelEncoder()
                df_clean[col] = le.fit_transform(df_clean[col])
                encoders[col] = le

    # Additionally, if any numeric column (not already marked as categorical) has only 2 unique values,
    # treat it as categorical and encode it.
    for col in df_clean.columns:
        if col not in categorical_cols and df_clean[col].nunique() == 2:
            categorical_cols.append(col)
            le = LabelEncoder()
            df_clean[col] = le.fit_transform(df_clean[col])
            encoders[col] = le

    # Continuous columns are those not marked as categorical or discrete.
    continuous_cols = [col for col in df_clean.columns if col not in categorical_cols + discrete_cols]

    return continuous_cols, discrete_cols, categorical_cols, df_clean, encoders

def reverse_encoding(df: pd.DataFrame, encoders: dict):
    """
    Reverse the LabelEncoder transformation on categorical columns.

    Parameters:
      df (pd.DataFrame): DataFrame with encoded categorical columns.
      encoders (dict): Dictionary mapping column names to their LabelEncoder.

    Returns:
      pd.DataFrame: A new DataFrame with the categorical columns decoded to their original labels.
    """
    df_decoded = df.copy()
    for col, le in encoders.items():
        # Ensure that the column is integer type before inverse transforming.
        df_decoded[col] = le.inverse_transform(df_decoded[col].astype(int))
    return df_decoded

df.drop('نامونامخانوادگی', axis=1,inplace=True)
cont, dis, cat, df2, enc = prep(df)


In [4]:
def create_missings(df:pd.DataFrame, missingness:float, random_seed:float=96):
    # Create random missingness.
    np.random.seed(random_seed)
    mask = np.random.rand(*df.shape) < (missingness / 100)
    mask_df = pd.DataFrame(mask, columns=df.columns)
    df_missing = df.mask(mask)
    return df_missing, mask_df


def calculate_metrics(df_clean, mask_df, imputed_df, method_name, continuous_cols, discrete_cols, categorical_cols):    
    mae_list = []
    for col in continuous_cols + discrete_cols:
        col_mask = mask_df[col]
        if col_mask.sum() > 0:
            error = np.abs(imputed_df.loc[col_mask, col] - df_clean.loc[col_mask, col])
            mae_list.append(error.mean())
    overall_mae = np.mean(mae_list) if mae_list else np.nan
    
    acc_list = []
    for col in categorical_cols:
        col_mask = mask_df[col]
        if col_mask.sum() > 0:
            acc = (imputed_df.loc[col_mask, col] == df_clean.loc[col_mask, col]).mean()
            acc_list.append(acc)
    overall_acc = np.mean(acc_list) if acc_list else np.nan

    return {'Method': method_name, 'MAE': overall_mae, 'Accuracy': overall_acc}


df3, df3_mask = create_missings(df2,20)
df3

,side,gccaverarage1,supgcc1,infgcc1,intraeye1,flv1,glv1,wholethick1,thicksuphemi1,thickinfhem1,...,deepvdinf1_injured_D,SSI1_injured,FAZ1_injured,PERIM1_injured,FD1_injured,N1000_injured,N500_injured,CENTR_injured,T500_injured,T1000_injured
0,1.0,98.0,96.0,100.0,-4.0,NaN,NaN,278.0,280.0,NaN,...,41.70,NaN,0.361,NaN,48.28,282.0,266.0,256.0,253.0,NaN
1,NaN,105.0,104.0,106.0,-2.0,0.04,0.07,314.0,314.0,315.0,...,43.20,0.6,0.260,1.911,54.63,389.0,392.0,416.0,395.0,386.0
2,1.0,101.0,102.0,NaN,2.0,0.27,0.45,284.0,286.0,NaN,...,28.60,0.7,0.262,2.045,54.68,185.0,NaN,194.0,179.0,170.0
3,NaN,90.0,88.0,92.0,-4.0,NaN,NaN,274.0,NaN,274.0,...,39.90,0.3,1.044,4.173,29.61,247.0,272.0,285.0,277.0,NaN
4,0.0,105.0,105.0,NaN,1.0,NaN,0.84,NaN,288.0,295.0,...,2.90,0.1,0.304,NaN,3.86,285.0,315.0,341.0,307.0,294.0
5,1.0,106.0,103.0,NaN,-5.0,0.01,0.10,NaN,310.0,NaN,...,33.60,NaN,0.244,1.920,49.63,204.0,210.0,225.0,208.0,193.0
6,0.0,108.0,107.0,109.0,-2.0,0.23,NaN,285.0,288.0,283.0,...,49.50,NaN,NaN,2.029,52.78,176.0,222.0,241.0,205.0,167.0
7,0.0,92.0,92.0,NaN,0.0,0.26,3.35,281.0,284.0,279.0,...,35.50,NaN,0.148,1.478,27.84,185.0,217.0,233.0,193.0,159.0
8,1.0,103.0,100.0,106.0,-6.0,0.05,0.29,294.0,294.0,NaN,...,50.10,0.4,0.151,1.468,58.54,182.0,198.0,207.0,173.0,153.0
9,1.0,95.0,NaN,NaN,-2.0,NaN,NaN,282.0,NaN,277.0,...,57.60,0.8,0.374,2.312,59.15,336.0,352.0,386.0,NaN,339.0


In [ ]:
# # Convert categorical columns and get categorical metadata.
# md_cat_data, md_cats = md.cat_conv(df3[cat])

# # Separate non-categorical (numeric) columns.
# num_cols = dis + cont  # these are the numeric columns

# # Drop the categorical columns from df3 and then combine with the converted categorical data.
# df3.drop(cat, axis=1, inplace=True)
# constructor_list = [df3]
# constructor_list.append(md_cat_data)
# data_in = pd.concat(constructor_list, axis=1)

# # Scale non-categorical columns BEFORE imputation.
# scaler = StandardScaler()
# data_in[num_cols] = scaler.fit_transform(data_in[num_cols])

# # Handle missing values (if needed).
# na_loc = data_in.isnull()
# data_in[na_loc] = np.nan

# # Build and train the imputer using the scaled data.
# imputer = md.Midas(layer_structure=[256, 256], vae_layer=False, seed=89, input_drop=0.75)
# # Note: Using the scaled data_in (you might want to assign it to a new variable like data_in2 if needed)
# imputer.build_model(data_in, softmax_columns=md_cats)
# imputer.train_model(training_epochs=20)

# # Generate imputations.
# imputations = imputer.generate_samples(m=10).output_list 

# # Reverse transform the numerical columns in each imputed DataFrame.
# for i in range(len(imputations)):
#     imputations[i][num_cols] = scaler.inverse_transform(imputations[i][num_cols])
    
#     # Process categorical columns:
#     tmp_cat = [imputations[i][x].idxmax(axis=1) for x in md_cats]
#     cat_df = pd.DataFrame({cat[j]: tmp_cat[j] for j in range(len(cat))})
#     # Construct final imputed dataset by reattaching categorical data.
#     flat_cats = [col for variable in md_cats for col in variable]
#     imputations[i] = pd.concat([imputations[i], cat_df], axis=1).drop(flat_cats, axis=1)


Size index: [138, 2]

Computation graph constructed

Model initialised

Epoch: 0 , loss: 102.03511810302734
Epoch: 1 , loss: 119.28441619873047
Epoch: 2 , loss: 125.48267364501953
Epoch: 3 , loss: 116.40061950683594
Epoch: 4 , loss: 96.99713897705078
Epoch: 5 , loss: 103.977783203125
Epoch: 6 , loss: 96.29216766357422
Epoch: 7 , loss: 98.9549331665039
Epoch: 8 , loss: 93.18431091308594
Epoch: 9 , loss: 114.44400024414062
Epoch: 10 , loss: 98.850830078125
Epoch: 11 , loss: 123.07731628417969
Epoch: 12 , loss: 126.85834503173828
Epoch: 13 , loss: 115.41191101074219
Epoch: 14 , loss: 110.39813232421875
Epoch: 15 , loss: 95.76268005371094
Epoch: 16 , loss: 102.50088500976562
Epoch: 17 , loss: 109.4853286743164
Epoch: 18 , loss: 114.66378021240234
Epoch: 19 , loss: 94.5094223022461
Training complete. Saving file...
Model saved in file: tmp/MIDAS
INFO:tensorflow:Restoring parameters from tmp/MIDAS
Model restored.


In [ ]:
# md_cats

[['side_0.0', 'side_1.0']]

In [ ]:
# soft_cols = [x for sublist in md_cats for x in sublist] + dis
# soft_cols

['side_0.0',
 'side_1.0',
 'gccaverarage1',
 'supgcc1',
 'infgcc1',
 'intraeye1',
 'wholethick1',
 'thicksuphemi1',
 'thickinfhem1',
 'thickfovea1',
 'thickparafove1',
 'thicksuphem1',
 'thickinfhemi1',
 'thicktempo1',
 'thicksup1',
 'thicknasal1',
 'thickinf1',
 'thickperi1',
 'thicksupe1',
 'thickinf1_A',
 'thicktempo1_A',
 'thicksup1_A',
 'thicknasal1_A',
 'thickinf1_B',
 'N1000',
 'N500',
 'CENTR',
 'T500',
 'T1000',
 'gccaverarage1_injured',
 'supgcc1_injured',
 'infgcc1_injured',
 'intraeye1_injured',
 'wholethick1_injured',
 'thicksuphemi1_injured',
 'thickinfhem1_injured',
 'thickfovea1_injured',
 'thickparafove1_injured',
 'thicksuphem1_injured',
 'thickinfhemi1_injured',
 'thicktempo1_injured',
 'thicksup1_injured',
 'thicknasal1_injured',
 'thickinf1_injured',
 'thickperi1_injured',
 'thicksupe1_injured',
 'thickinf1_injured_A',
 'thicktempo1_injured_A',
 'thicksup1_injured_A',
 'thicknasal1_injured_A',
 'thickinf1_injured_B',
 'N1000_injured',
 'N500_injured',
 'CENTR_injur

In [ ]:
# for i in imputations:
#     print(calculate_metrics(df2, df3_mask, i, f'midaspy ', cont, dis, cat))

{'Method': 'midaspy ', 'MAE': 13.964243235010507, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.074331916384256, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.04651589316078, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.008804357996462, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.059403184750666, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.061973330428705, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.04347700381949, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 13.98978603077068, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.039249661563625, 'Accuracy': 0.0}
{'Method': 'midaspy ', 'MAE': 14.032228158667737, 'Accuracy': 0.0}


In [ ]:
# imputations[1]

,gccaverarage1,supgcc1,infgcc1,intraeye1,flv1,glv1,wholethick1,thicksuphemi1,thickinfhem1,thickfovea1,...,SSI1_injured,FAZ1_injured,PERIM1_injured,FD1_injured,N1000_injured,N500_injured,CENTR_injured,T500_injured,T1000_injured,side
0,98.000000,96.000000,100.000000,-4.000000e+00,0.087689,1.809129,278.000000,280.000000,279.407634,240.000000,...,0.565862,0.361000,2.565016,48.280000,282.000000,266.000000,256.000000,253.000000,254.869400,side_1.0
1,105.000000,104.000000,106.000000,-2.000000e+00,0.040000,0.070000,314.000000,314.000000,315.000000,252.000000,...,0.600000,0.260000,1.911000,54.630000,389.000000,392.000000,416.000000,395.000000,386.000000,side_0.0
2,101.000000,102.000000,101.876611,2.000000e+00,0.270000,0.450000,284.000000,286.000000,279.554361,241.000000,...,0.700000,0.262000,2.045000,54.680000,185.000000,273.925071,194.000000,179.000000,170.000000,side_1.0
3,90.000000,88.000000,92.000000,-4.000000e+00,0.126120,2.116542,274.000000,284.443301,274.000000,235.000000,...,0.300000,1.044000,4.173000,29.610000,247.000000,272.000000,285.000000,277.000000,254.770206,side_0.0
4,105.000000,105.000000,102.184424,1.000000e+00,0.070173,0.840000,281.265541,288.000000,295.000000,323.000000,...,0.100000,0.304000,2.618244,3.860000,285.000000,315.000000,341.000000,307.000000,294.000000,side_0.0
5,106.000000,103.000000,101.799055,-5.000000e+00,0.010000,0.100000,281.081961,310.000000,280.481379,286.000000,...,0.567203,0.244000,1.920000,49.630000,204.000000,210.000000,225.000000,208.000000,193.000000,side_1.0
6,108.000000,107.000000,109.000000,-2.000000e+00,0.230000,1.760781,285.000000,288.000000,283.000000,243.000000,...,0.572085,0.492184,2.029000,52.780000,176.000000,222.000000,241.000000,205.000000,167.000000,side_0.0
7,92.000000,92.000000,101.955636,2.220446e-16,0.260000,3.350000,281.000000,284.000000,279.000000,258.000000,...,0.571178,0.148000,1.478000,27.840000,185.000000,217.000000,233.000000,193.000000,159.000000,side_0.0
8,103.000000,100.000000,106.000000,-6.000000e+00,0.050000,0.290000,294.000000,294.000000,281.151862,273.000000,...,0.400000,0.151000,1.468000,58.540000,182.000000,198.000000,207.000000,173.000000,153.000000,side_1.0
9,95.000000,99.905187,101.774283,-2.000000e+00,0.054704,1.807580,282.000000,282.463267,277.000000,245.000000,...,0.800000,0.374000,2.312000,59.150000,336.000000,352.000000,386.000000,267.204811,339.000000,side_1.0


In [10]:
# def do_midas(df, continuous_cols=None, discrete_cols=None, categorical_cols=None):

#     # Convert categorical columns and get categorical metadata.
#     md_cat_data, md_cats = md.cat_conv(df[categorical_cols])

#     # Separate non-categorical (numeric) columns.
#     num_cols = discrete_cols + continuous_cols  # these are the numeric columns

#     # Drop the categorical columns from df3 and then combine with the converted categorical data.
    
#     df_copy = df.drop(cat, axis=1)
#     constructor_list = [df_copy]
#     constructor_list.append(md_cat_data)
#     data_in = pd.concat(constructor_list, axis=1)

#     # Scale non-categorical columns BEFORE imputation.
#     scaler = StandardScaler()
#     data_in[num_cols] = scaler.fit_transform(data_in[num_cols])

#     # Handle missing values (if needed).
#     na_loc = data_in.isnull()
#     data_in[na_loc] = np.nan

#     # Build and train the imputer using the scaled data.
#     imputer = md.Midas(layer_structure=[256, 256], vae_layer=True, seed=89, input_drop=0.75)
    
#     # Note: Using the scaled data_in (you might want to assign it to a new variable like data_in2 if needed)
#     # soft_cols = [x for sublist in md_cats for x in sublist] + dis

#     # imputer.build_model(data_in, softmax_columns=[soft_cols])
#     imputer.build_model(data_in, softmax_columns=md_cats)

#     imputer.train_model(training_epochs=20)

#     # Generate imputations.
#     imps = imputer.generate_samples(m=10).output_list 

#     # Reverse transform the numerical columns in each imputed DataFrame.
#     for i in imps:
#         i[num_cols] = scaler.inverse_transform(i[num_cols])
        
#         # Process categorical columns:
#         tmp_cat = [i[x].idxmax(axis=1) for x in md_cats]
#         cat_df = pd.DataFrame({cat[j]: tmp_cat[j] for j in range(len(cat))})
#         # Construct final imputed dataset by reattaching categorical data.
#         flat_cats = [col for variable in md_cats for col in variable]
#         i = pd.concat([i, cat_df], axis=1).drop(flat_cats, axis=1)
#         # i[discrete_cols] = i[discrete_cols].round()
        
#     return imps 
        

In [11]:
def do_midas(df, continuous_cols=None, discrete_cols=None, categorical_cols=None,
              layer:list=[256,256], vae:bool=True, samples:int=10 ):
    """
    Imputes missing values using the MIDAS model.
    
    Parameters:
      df (pd.DataFrame): Input dataframe.
      continuous_cols (list): List of continuous column names.
      discrete_cols (list): List of discrete (numeric but non-continuous) column names.
      categorical_cols (list): List of categorical column names.
      
    Returns:
      imps (list): A list of imputed dataframes.
    """
    # 1. Convert categorical columns and get categorical metadata.
    md_cat_data, md_cats = md.cat_conv(df[categorical_cols])
    
    # 2. Define the numeric columns.
    num_cols = discrete_cols + continuous_cols  # these are the numeric columns

    # 3. Drop original categorical columns and combine with the converted categorical data.
    df_copy = df.drop(columns=categorical_cols)
    constructor_list = [df_copy, md_cat_data]
    data_in = pd.concat(constructor_list, axis=1)
    
    # 4. Scale non-categorical columns BEFORE imputation.
    scaler = StandardScaler()
    data_in[num_cols] = scaler.fit_transform(data_in[num_cols])
    
    # (Optional) Handle missing values if needed.
    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan
    
    # 5. Build and train the imputer using the scaled data.
    imputer = md.Midas(layer_structure=layer, vae_layer=vae, seed=96, input_drop=0.75)
    # Use md_cats as softmax columns for categorical outputs.
    imputer.build_model(data_in, softmax_columns=md_cats)
    imputer.train_model(training_epochs=20)
    
    # 6. Generate imputations.
    imps = imputer.generate_samples(m=samples).output_list
    
    # 7. Post-process each imputed DataFrame.
    for idx, imp_df in enumerate(imps):
        # Reverse transform the numeric columns.
        imp_df[num_cols] = scaler.inverse_transform(imp_df[num_cols])
        
        # Process categorical columns.
        # For each softmax group in md_cats, choose the column with the highest probability.
        tmp_cat = []
        for group in md_cats:
            # idxmax returns the column name with maximum value per row for this group.
            tmp_cat.append(imp_df[group].idxmax(axis=1))
        # Assume the order of md_cats corresponds to categorical_cols.
        cat_df = pd.DataFrame({categorical_cols[j]: tmp_cat[j] for j in range(len(categorical_cols))})
        
        # Drop the softmax columns.
        flat_cats = [col for group in md_cats for col in group]
        imp_df = pd.concat([imp_df, cat_df], axis=1).drop(columns=flat_cats)
        
        # Handle discrete data by rounding the values.
        imp_df[discrete_cols] = imp_df[discrete_cols].round()
        
        # Replace the processed DataFrame in the list.
        imps[idx] = imp_df

        ### make method info
        method_info = f'MIDAS, params: samples={samples} ,layer={layer}, vae={vae}'
    return imps, method_info

In [ ]:
def simulate_missingness(df, show_missingness=False):
    """
    Takes a DataFrame, calculates missingness for each column, drops all rows with any missing values (df2),
    then reintroduces missing values to df2 to match the original missingness proportions, resulting in df3.
    Also returns a mask of artificial missing values.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        show_missingness (bool): If True, prints the missingness percentage for each column 
                                 in the original DataFrame and in the simulated DataFrame.
    
    Returns:
        tuple: A tuple (df3, artificial_mask) where:
            - df3 (pd.DataFrame): A new DataFrame with simulated missingness.
            - artificial_mask (pd.DataFrame): A boolean mask indicating the positions where missing values were artificially inserted.
    """
    # 1. Calculate original missingness fraction for each column.
    missing_original = df.isna().mean()
    
    # 2. Drop all rows with missing values to create df2.
    df2 = df.dropna().reset_index(drop=True)
    
    # 3. Create df3 by copying df2.
    df3 = df2.copy()
    
    # Create a mask DataFrame with the same shape as df3 to mark artificial missing values.
    missing_mask = pd.DataFrame(False, index=df3.index, columns=df3.columns)
    
    # 4. Reintroduce missing values in df3 based on the original missingness proportions.
    for col in df3.columns:
        # Calculate the number of entries to set as missing in this column.
        n_missing = int(round(missing_original[col] * len(df3)))
        if n_missing > 0:
            # Randomly select indices to set as missing.
            missing_indices = df3.sample(n=n_missing, random_state=42).index
            df3.loc[missing_indices, col] = np.nan
            missing_mask.loc[missing_indices, col] = True

    # 5. Optionally print missingness for each column.
    if show_missingness:
        missing_df3 = df3.isna().mean()
        print("Missingness Comparison:")
        for col in df.columns:
            print(f"Column '{col}': Original: {missing_original[col]*100:.2f}%  \t -> \t df3: {missing_df3[col]*100:.2f}%")
    
    # Return the simulated DataFrame and the mask.
    return df2, df3, missing_mask

full_df, df4, df4_mask = simulate_missingness(df, show_missingness=True)


Missingness Comparison:
Column 'side': Original: 0.00%  	 -> 	 df3: 0.00%
Column 'gccaverarage1': Original: 0.00%  	 -> 	 df3: 0.00%
Column 'supgcc1': Original: 0.00%  	 -> 	 df3: 0.00%
Column 'infgcc1': Original: 0.00%  	 -> 	 df3: 0.00%
Column 'intraeye1': Original: 0.00%  	 -> 	 df3: 0.00%
Column 'flv1': Original: 11.11%  	 -> 	 df3: 9.68%
Column 'glv1': Original: 11.11%  	 -> 	 df3: 9.68%
Column 'wholethick1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thicksuphemi1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thickinfhem1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thickfovea1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thickparafove1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thicksuphem1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thickinfhemi1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thicktempo1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thicksup1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thicknasal1': Original: 4.44%  	 -> 	 df3: 3.23%
Column 'thickinf1': Origina

In [13]:
imps,info = do_midas(df4, cont, dis, cat)

Size index: [138, 2]
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.

Computation graph constructed

Model initialised

Epoch: 0 , loss: 114.55461120605469
Epoch: 1 , loss: 144.34469604492188
Epoch: 2 , loss: 124.42644500732422
Epoch: 3 , loss: 148.06011962890625
Epoch: 4 , loss: 134.1834716796875
Epoch: 5 , loss: 137.03042602539062
Epoch: 6 , loss: 140.19143676757812
Epoch: 7 , loss: 138.25662231445312
Epoch: 8 , loss: 115.105712890625
Epoch: 9 , loss: 131.341796875
Epoch: 10 , loss: 152.06472778320312
Epoch: 11 , loss: 114.97909545898438
Epoch: 12 , loss: 119.94746398

In [14]:
imps[3]

,gccaverarage1,supgcc1,infgcc1,intraeye1,flv1,glv1,wholethick1,thicksuphemi1,thickinfhem1,thickfovea1,...,SSI1_injured,FAZ1_injured,PERIM1_injured,FD1_injured,N1000_injured,N500_injured,CENTR_injured,T500_injured,T1000_injured,side
0,98.0,96.0,100.0,-4.0,0.710000,0.960000,278.0,280.0,275.0,240.0,...,0.600000,0.361000,2.529000,48.280000,282.0,266.0,256.0,253.0,244.0,side_OS
1,105.0,104.0,106.0,-2.0,0.040000,0.070000,314.0,314.0,315.0,252.0,...,0.600000,0.260000,1.911000,54.630000,389.0,392.0,416.0,395.0,386.0,side_OS
2,101.0,102.0,100.0,2.0,0.270000,0.450000,284.0,286.0,283.0,241.0,...,0.700000,0.262000,2.045000,54.680000,185.0,182.0,194.0,179.0,170.0,side_OS
3,90.0,88.0,92.0,-4.0,0.000000,4.980000,274.0,274.0,274.0,235.0,...,0.300000,1.044000,4.173000,29.610000,247.0,272.0,285.0,277.0,261.0,side_OD
4,105.0,105.0,104.0,1.0,0.830000,0.840000,291.0,288.0,295.0,323.0,...,0.100000,0.304000,2.271000,3.860000,285.0,315.0,341.0,307.0,294.0,side_OD
5,106.0,103.0,108.0,-5.0,0.010000,0.100000,309.0,310.0,308.0,286.0,...,0.700000,0.244000,1.920000,49.630000,204.0,210.0,225.0,208.0,193.0,side_OS
6,108.0,107.0,109.0,-2.0,0.230000,0.240000,285.0,288.0,283.0,243.0,...,0.900000,0.286000,2.029000,52.780000,176.0,222.0,241.0,205.0,167.0,side_OD
7,92.0,92.0,92.0,0.0,0.260000,3.350000,281.0,284.0,279.0,258.0,...,0.500000,0.148000,1.478000,27.840000,185.0,217.0,233.0,193.0,159.0,side_OD
8,103.0,100.0,106.0,-6.0,0.050000,0.290000,294.0,294.0,293.0,273.0,...,0.400000,0.151000,1.468000,58.540000,182.0,198.0,207.0,173.0,153.0,side_OS
9,95.0,94.0,96.0,-2.0,0.120000,2.820000,282.0,287.0,277.0,245.0,...,0.800000,0.374000,2.312000,59.150000,336.0,352.0,386.0,345.0,339.0,side_OS


In [15]:

# def select_best_imputations(imputed_dfs, original_df, mask_df, continuous_cols, discrete_cols, categorical_cols, method_names=None):
   
#     n_methods = len(imputed_dfs)
#     if method_names is None:
#         method_names = [f"Method {i+1}" for i in range(n_methods)]
    
#     # Dictionary to store best method index per column.
#     best_method_per_col = {}
#     summary_list = []
    
#     # Iterate through each column.
#     for col in original_df.columns:
#         # Determine the data type based on provided lists.
#         if col in continuous_cols:
#             col_data_type = "Continuous"
#         elif col in discrete_cols:
#             col_data_type = "Discrete"
#         elif col in categorical_cols:
#             col_data_type = "Categorical"
#         else:
#             col_data_type = str(original_df[col].dtype)
        
#         # Only evaluate columns that had artificial missing values.
#         if mask_df[col].sum() == 0:
#             best_method_per_col[col] = None
#             summary_list.append({
#                 'Column': col,
#                 'Data Type': col_data_type,
#                 'Best Method': None,
#                 'Metric': np.nan
#             })
#             continue
        
#         # List to store metrics for each method for this column.
#         metrics = []
        
#         if col in continuous_cols or col in discrete_cols:
#             for df_imp in imputed_dfs:
#                 # Using bracket notation for boolean indexing.
#                 error = np.abs(df_imp[col][mask_df[col]] - original_df[col][mask_df[col]])
#                 mae = error.mean() if not error.empty else np.nan
#                 metrics.append(mae)
#             best_idx = np.nanargmin(metrics)
#             best_metric = metrics[best_idx]
#         elif col in categorical_cols:
#             for df_imp in imputed_dfs:
#                 correct = (df_imp[col][mask_df[col]] == original_df[col][mask_df[col]])
#                 acc = correct.mean() if not correct.empty else np.nan
#                 metrics.append(acc)
#             best_idx = np.nanargmax(metrics)
#             best_metric = metrics[best_idx]
#         else:
#             best_idx = None
#             best_metric = np.nan
        
#         best_method = method_names[best_idx] if best_idx is not None else None
#         best_method_per_col[col] = best_idx
        
#         summary_list.append({
#             'Column': col,
#             'Data Type': col_data_type,
#             'Best Method': best_method,
#             'Metric': best_metric
#         })
    
#     summary_table = pd.DataFrame(summary_list)
    
#     # Build best-imputed dataframe by replacing only the masked entries with the best imputed values.
#     best_imputed_df = original_df.copy()
#     for col in original_df.columns:
#         if mask_df[col].sum() > 0 and best_method_per_col[col] is not None:
#             method_idx = best_method_per_col[col]
#             best_imputed_df.loc[mask_df[col], col] = imputed_dfs[method_idx].loc[mask_df[col], col]
    
#     return best_imputed_df, summary_table


In [16]:
def select_best_imputations(imputed_dfs, original_df, mask_df, continuous_cols, discrete_cols, categorical_cols, method_info=None, method_names=None):
    
    n_methods = len(imputed_dfs)
    
    # If method_info is provided, split it to get the method name and parameters.
    if method_info is not None:
        parts = method_info.split(',')
        extracted_method_name = parts[0].strip()
        params = ','.join(parts[1:]).strip() if len(parts) > 1 else ""
        method_names = [f"{extracted_method_name} ({params})"] * n_methods
    elif method_names is None:
        method_names = [f"Method {i+1}" for i in range(n_methods)]
    
    # Dictionary to store best method index per column.
    best_method_per_col = {}
    summary_list = []
    
    # Iterate through each column.
    for col in original_df.columns:
        # Determine the data type based on provided lists.
        if col in continuous_cols:
            col_data_type = "Continuous"
        elif col in discrete_cols:
            col_data_type = "Discrete"
        elif col in categorical_cols:
            col_data_type = "Categorical"
        else:
            col_data_type = str(original_df[col].dtype)
        
        # Only evaluate columns that had artificial missing values.
        if mask_df[col].sum() == 0:
            best_method_per_col[col] = None
            summary_list.append({
                'Column': col,
                'Data Type': col_data_type,
                'Best Method': None,
                'Metric': np.nan
            })
            continue
        
        # List to store metrics for each method for this column.
        metrics = []
        
        if col in continuous_cols or col in discrete_cols:
            for df_imp in imputed_dfs:
                # Compute MAE only over the artificially masked rows.
                error = np.abs(df_imp[col][mask_df[col]] - original_df[col][mask_df[col]])
                mae = error.mean() if not error.empty else np.nan
                metrics.append(mae)
            best_idx = np.nanargmin(metrics)
            best_metric = metrics[best_idx]
        elif col in categorical_cols:
            for df_imp in imputed_dfs:
                correct = (df_imp[col][mask_df[col]] == original_df[col][mask_df[col]])
                acc = correct.mean() if not correct.empty else np.nan
                metrics.append(acc)
            best_idx = np.nanargmax(metrics)
            best_metric = metrics[best_idx]
        else:
            best_idx = None
            best_metric = np.nan
        
        best_method = method_names[best_idx] if best_idx is not None else None
        best_method_per_col[col] = best_idx
        
        summary_list.append({
            'Column': col,
            'Data Type': col_data_type,
            'Best Method': best_method,
            'Metric': best_metric
        })
    
    summary_table = pd.DataFrame(summary_list)
    
    # Build best-imputed dataframe by replacing only the masked entries with the best imputed values.
    best_imputed_df = original_df.copy()
    for col in original_df.columns:
        if mask_df[col].sum() > 0 and best_method_per_col[col] is not None:
            method_idx = best_method_per_col[col]
            best_imputed_df.loc[mask_df[col], col] = imputed_dfs[method_idx].loc[mask_df[col], col]
    
    return best_imputed_df, summary_table


In [17]:
best, summary = select_best_imputations(imps, full_df, df4_mask,cont, dis, cat, method_info=info)

In [18]:
summary

,Column,Data Type,Best Method,Metric
0,side,Categorical,None,NaN
1,gccaverarage1,Discrete,None,NaN
2,supgcc1,Discrete,None,NaN
3,infgcc1,Discrete,None,NaN
4,intraeye1,Discrete,None,NaN
...,...,...,...,...
134,N1000_injured,Discrete,"MIDAS (params: samples=10 ,layer=[256, 256], v...",13.0
135,N500_injured,Discrete,"MIDAS (params: samples=10 ,layer=[256, 256], v...",12.0
136,CENTR_injured,Discrete,"MIDAS (params: samples=10 ,layer=[256, 256], v...",7.0
137,T500_injured,Discrete,"MIDAS (params: samples=10 ,layer=[256, 256], v...",0.0


In [19]:
best[dis] = best[dis].round()
best

,side,gccaverarage1,supgcc1,infgcc1,intraeye1,flv1,glv1,wholethick1,thicksuphemi1,thickinfhem1,...,deepvdinf1_injured_D,SSI1_injured,FAZ1_injured,PERIM1_injured,FD1_injured,N1000_injured,N500_injured,CENTR_injured,T500_injured,T1000_injured
0,OS,98,96,100,-4,0.710000,0.960000,278.0,280.0,275.0,...,41.700000,0.600000,0.361000,2.529000,48.280000,282.0,266.0,256.0,253.0,244.0
1,OS,105,104,106,-2,0.040000,0.070000,314.0,314.0,315.0,...,43.200000,0.600000,0.260000,1.911000,54.630000,389.0,392.0,416.0,395.0,386.0
2,OS,101,102,100,2,0.270000,0.450000,284.0,286.0,283.0,...,28.600000,0.700000,0.262000,2.045000,54.680000,185.0,182.0,194.0,179.0,170.0
3,OD,90,88,92,-4,0.000000,4.980000,274.0,274.0,274.0,...,39.900000,0.300000,1.044000,4.173000,29.610000,247.0,272.0,285.0,277.0,261.0
4,OD,105,105,104,1,0.830000,0.840000,291.0,288.0,295.0,...,2.900000,0.100000,0.304000,2.271000,3.860000,285.0,315.0,341.0,307.0,294.0
5,OS,106,103,108,-5,0.010000,0.100000,309.0,310.0,308.0,...,33.600000,0.700000,0.244000,1.920000,49.630000,204.0,210.0,225.0,208.0,193.0
6,OD,108,107,109,-2,0.230000,0.240000,285.0,288.0,283.0,...,49.500000,0.900000,0.286000,2.029000,52.780000,176.0,222.0,241.0,205.0,167.0
7,OD,92,92,92,0,0.260000,3.350000,281.0,284.0,279.0,...,35.500000,0.500000,0.148000,1.478000,27.840000,185.0,217.0,233.0,193.0,159.0
8,OS,103,100,106,-6,0.050000,0.290000,294.0,294.0,293.0,...,50.100000,0.400000,0.151000,1.468000,58.540000,182.0,198.0,207.0,173.0,153.0
9,OS,95,94,96,-2,0.120000,2.820000,282.0,287.0,277.0,...,57.600000,0.800000,0.374000,2.312000,59.150000,336.0,352.0,386.0,345.0,339.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

def knn_impute_df(df, continuous_cols=None, discrete_cols=None, categorical_cols=None, n_neighbors=5, samples=1):
    """
    Impute missing values in a DataFrame using KNN imputation.
    
    Assumes:
      - Continuous columns are numeric.
      - Discrete columns are numeric and integer-like.
      - Categorical columns have been label encoded (with missing values represented as np.nan).
    
    Parameters:
      df (pd.DataFrame): Input DataFrame with missing values.
      continuous_cols (list of str): Names of continuous numeric columns.
      discrete_cols (list of str): Names of discrete numeric columns.
      categorical_cols (list of str): Names of categorical columns.
      n_neighbors (int): Number of neighbors for KNN imputation.
      samples (int): Number of imputed DataFrame samples to return.
    
    Returns:
      tuple: (imps, method_info)
          - imps (list of pd.DataFrame): List of imputed DataFrames.
          - method_info (str): String with method information (e.g., "KNN, params: n_neighbors=5").
    """
    # Work on a copy of the dataframe
    df_imputed = df.copy()
    
    # Impute continuous columns
    if continuous_cols:
        imputer_cont = KNNImputer(n_neighbors=n_neighbors)
        df_imputed[continuous_cols] = imputer_cont.fit_transform(df_imputed[continuous_cols])
    
    # Impute discrete columns and round to integer
    if discrete_cols:
        imputer_disc = KNNImputer(n_neighbors=n_neighbors)
        imputed_disc = imputer_disc.fit_transform(df_imputed[discrete_cols])
        df_imputed[discrete_cols] = np.round(imputed_disc).astype(int)
    
    # Impute categorical columns (assumed to be label encoded)
    if categorical_cols:
        imputer_cat = KNNImputer(n_neighbors=n_neighbors)
        imputed_cat = imputer_cat.fit_transform(df_imputed[categorical_cols])
        df_imputed[categorical_cols] = np.round(imputed_cat).astype(int)
    
    # Replicate the imputed DataFrame 'samples' times.
    imps = [df_imputed]
    
    # Build method_info string.
    method_info = f"KNN, params: n_neighbors={n_neighbors}"
    
    return imps, method_info


In [21]:
def mice_forest_impute_df(df, continuous_cols=None, discrete_cols=None, categorical_cols=None, 
                          iters=10, strat='normal', samples=1):
    """
    Impute missing values in a DataFrame using the MICE forest method.
    
    Assumes:
      - Continuous columns are numeric.
      - Discrete columns are numeric and integer-like.
      - Categorical columns have been label encoded (with missing values represented as np.nan).
    
    Parameters:
      df (pd.DataFrame): Input DataFrame with missing values.
      continuous_cols (list of str): Names of continuous numeric columns.
      discrete_cols (list of str): Names of discrete numeric columns.
      categorical_cols (list of str): Names of categorical columns.
      iters (int): Number of MICE iterations.
      strat: ['normal', 'shap', 'fast'] or a dictionary specifying the mean matching strategy.
      samples (int): Number of imputed DataFrame samples to return.
    
    Returns:
      tuple: (imps, method_info)
          - imps (list of pd.DataFrame): List of imputed DataFrames.
          - method_info (str): A string with method information, e.g., 
              "MICE Forest, params: iters=10, strat=shap"
    """
    # Create a copy of the DataFrame to avoid modifying the original data.
    df_imputed = df.copy()
    
    # Create an imputation kernel using miceforest.
    kernel = mf.ImputationKernel(
        df_imputed,
        random_state=0, 
        mean_match_strategy=strat
    )
    
    # Run the MICE algorithm for the specified number of iterations.
    kernel.mice(iterations=iters)
    
    # Retrieve the completed data (imputed dataset).
    df_completed = kernel.complete_data(dataset=0)
    
    # For discrete and categorical columns, round the imputed values to integers.
    if discrete_cols:
        df_completed[discrete_cols] = df_completed[discrete_cols].round().astype(int)
    if categorical_cols:
        df_completed[categorical_cols] = df_completed[categorical_cols].round().astype(int)
    
    # Replicate the imputed DataFrame 'samples' times.
    imps = [df_completed]
    
    # Build the method info string.
    method_info = f"MICE Forest, params: iters={iters}, strat={strat}"
    
    return imps, method_info


In [22]:
def missforest_impute_df(df, continuous_cols=None, discrete_cols=None, categorical_cols=None, iters=5, samples=1):
    """
    Impute missing values in a DataFrame using the MissForest algorithm.
    
    Parameters:
      df (pd.DataFrame): Input DataFrame with missing values.
      continuous_cols (list of str): Names of continuous numeric columns.
      discrete_cols (list of str): Names of discrete numeric columns.
      categorical_cols (list of str): Names of categorical columns.
      iters (int): Maximum number of iterations for the MissForest algorithm.
      samples (int): Number of imputed DataFrame samples to return.
    
    Returns:
      tuple: (imps, method_info)
          - imps (list of pd.DataFrame): List of imputed DataFrames.
          - method_info (str): A string with method information, e.g., "MissForest, params: iters=5"
    """
    df_imputed = df.copy()
    
    # Create and run the MissForest imputer
    imputer = MissForest(max_iter=iters, categorical=categorical_cols)
    df_imputed_result = imputer.fit_transform(df_imputed)
    
    # For discrete and categorical columns, round the imputed values to integers.
    if discrete_cols:
        df_imputed_result[discrete_cols] = df_imputed_result[discrete_cols].round().astype(int)
    if categorical_cols:
        df_imputed_result[categorical_cols] = df_imputed_result[categorical_cols].round().astype(int)
    
    # Replicate the imputed DataFrame 'samples' times.
    imps = [df_imputed_result]
    
    # Build the method info string.
    method_info = f"MissForest, params: iters={iters}"
    
    return imps, method_info

# Example usage:
mf_imputed, mf_method_info = missforest_impute_df(df4, continuous_cols=cont, discrete_cols=dis, categorical_cols=cat)


c:\Users\Matin\AppData\Local\Programs\Python\Python310\lib\site-packages\missforest\missforest.py:333: UserWarning: Label encoding is no longer performed by default. Users will have to perform categorical features encoding by themselves.
  warnings.warn("Label encoding is no longer performed by default. "
  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\Matin\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Matin\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\Matin\AppData\Local\Programs\

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: side: object